In [ ]:
import time
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import talib

# === CONFIG FOR $9 CAPITAL ===
SYMBOLS = [
    "XAUUSD", "EURUSD", "GBPUSD", "USDJPY", "AUDUSD",
    "USDCAD", "NZDUSD", "USDCHF", "EURJPY", "GBPJPY",
    "EURGBP", "CADJPY", "CHFJPY", "AUDJPY", "EURNZD"
]
TIMEFRAME = mt5.TIMEFRAME_M15
LOT_SIZE = 0.01  # smallest lot for micro-trading
MIN_PROFIT_USD = 0.10  # close when even 10 cents profit is there
MAX_TRADES = 1  # limit risk due to small balance
FORCE_MODE = True  # always take a trade

# === INIT ===
if not mt5.initialize():
    raise RuntimeError(f"MT5 initialize() failed: {mt5.last_error()}")

account = mt5.account_info()
if account is None:
    raise RuntimeError(f"MT5 account_info() failed: {mt5.last_error()}")

for symbol in SYMBOLS:
    info = mt5.symbol_info(symbol)
    if info is None:
        raise RuntimeError(f"Symbol {symbol} not found")
    if not info.visible:
        mt5.symbol_select(symbol, True)

# === HELPER: Get historical candle data ===
def get_data(symbol, timeframe, n=100):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, n)
    if rates is None or len(rates) == 0:
        raise RuntimeError(f"Failed to get data for {symbol}")
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

# === Signal strategy using RSI + Engulfing ===
def get_trade_signal(df, symbol):
    close = df['close'].astype(float).values
    open_ = df['open'].astype(float).values
    high = df['high'].astype(float).values
    low = df['low'].astype(float).values
    rsi = talib.RSI(close, timeperiod=14)
    engulf = talib.CDLENGULFING(open_, high, low, close)

    if len(rsi) == 0 or len(engulf) == 0:
        return None

    last_rsi = rsi[-1]
    last_engulf = engulf[-1]
    print(f"{symbol} - RSI: {last_rsi:.2f}, Engulfing: {last_engulf}")

    if last_engulf > 0 and last_rsi < 40:
        return "buy"
    elif last_engulf < 0 and last_rsi > 60:
        return "sell"
    return None

# === Open trade ===
def open_trade(symbol, direction):
    tick = mt5.symbol_info_tick(symbol)
    price = tick.ask if direction == "buy" else tick.bid
    order_type = mt5.ORDER_TYPE_BUY if direction == "buy" else mt5.ORDER_TYPE_SELL

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": LOT_SIZE,
        "type": order_type,
        "price": price,
        "deviation": 20,
        "magic": 10001,
        "comment": "micro_profit_trade",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    result = mt5.order_send(request)
    if result.retcode == mt5.TRADE_RETCODE_DONE:
        print(f"✅ OPENED {direction.upper()} for {symbol} at {price}")
    else:
        print(f"❌ Order failed for {symbol}: {result.retcode}")
        print(result._asdict())

# === Close trade when profit >= $0.10 ===
def check_and_close():
    for symbol in SYMBOLS:
        positions = mt5.positions_get(symbol=symbol)
        if positions:
            pos = positions[0]
            profit = pos.profit
            print(f"{symbol} live profit: ${profit:.2f}")
            if profit >= MIN_PROFIT_USD:
                close_type = mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
                price = mt5.symbol_info_tick(symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(symbol).ask
                close_req = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": symbol,
                    "volume": pos.volume,
                    "type": close_type,
                    "position": pos.ticket,
                    "price": price,
                    "deviation": 20,
                    "magic": 10001,
                    "comment": "close_profit",
                    "type_time": mt5.ORDER_TIME_GTC,
                    "type_filling": mt5.ORDER_FILLING_IOC,
                }
                result = mt5.order_send(close_req)
                if result.retcode == mt5.TRADE_RETCODE_DONE:
                    print(f"✅ CLOSED {symbol} with profit ${profit:.2f}")
                else:
                    print(f"❌ Close failed: {result.retcode}")

# === MAIN LOOP ===
print("🚀 Running safe micro-profit bot for low capital ($9)...")
try:
    while True:
        check_and_close()
        time.sleep(2)

        if len(mt5.positions_get() or []) >= MAX_TRADES:
            print("⛔ Trade limit reached. Waiting...")
            time.sleep(60)
            continue

        for symbol in SYMBOLS:
            if not any(pos.symbol == symbol for pos in mt5.positions_get() or []):
                df = get_data(symbol, TIMEFRAME, 100)
                signal = get_trade_signal(df, symbol)
                open_trade(symbol, signal if signal else "buy")

        time.sleep(60)
except KeyboardInterrupt:
    print("🛑 Stopped by user")
finally:
    mt5.shutdown()


🚀 Running safe micro-profit bot for low capital ($9)...
XAUUSD live profit: $-17.80
EURUSD live profit: $-3.00
GBPUSD live profit: $-4.20
AUDUSD live profit: $-2.50
NZDUSD live profit: $-3.70
USDCHF live profit: $-3.52
EURJPY live profit: $-5.85
GBPJPY live profit: $-7.52
EURGBP live profit: $-3.08
CADJPY live profit: $-4.25
CHFJPY live profit: $-8.14
AUDJPY live profit: $-5.78
EURNZD live profit: $-3.21
⛔ Trade limit reached. Waiting...
XAUUSD live profit: $-16.90
EURUSD live profit: $-0.20
GBPUSD live profit: $-2.20
AUDUSD live profit: $-2.10
NZDUSD live profit: $-3.10
USDCHF live profit: $-6.07
EURJPY live profit: $-5.08
GBPJPY live profit: $-8.15
EURGBP live profit: $-2.15
CADJPY live profit: $-5.15
CHFJPY live profit: $-7.17
AUDJPY live profit: $-6.61
EURNZD live profit: $-1.90
⛔ Trade limit reached. Waiting...
XAUUSD live profit: $-18.70
GBPUSD live profit: $-1.10
AUDUSD live profit: $-1.70
NZDUSD live profit: $-3.20
USDCHF live profit: $-4.98
EURJPY live profit: $-5.15
GBPJPY li